In [ ]:
#Import the requisite library
import sys
sys.path.insert(0, '/mind_data/cdm_repos/cdm-utilities/')
sys.path.insert(0, '/mind_data/cdm_repos/cdm-utilities/minio_api')
import re

from data_classes_cdm import CDMProcessingVariables as config_cdm
import pandas as pd
import numpy as np
from minio_api import MinioAPI
from utils import set_debug_console, mrn_zero_pad, print_df_without_index, convert_to_int
%config Completer.use_jedi = False



In [ ]:
set_debug_console()

In [ ]:
fname_minio_env = config_cdm.minio_env
obj_minio = MinioAPI(fname_minio_env=fname_minio_env)


In [ ]:
_col_order = ['MRN', 'START_DATE', 'STOP_DATE', 'EVENT_TYPE', 'TREATMENT_TYPE', 'AGENT', 'THERAPY', 'SUBTYPE', 'RX_STATUS', 'RX_ROUTE', 'RX_INVESTIGATIVE']


In [ ]:
fname_pdl1 = 'pathology/pathology_pdl1_calls.tsv'
fname_gleason = 'pathology/pathology_gleason_calls.tsv'
fname_timeline_pdl1 = 'pathology/table_timeline_pdl1.tsv'
fname_timeline_gleason = 'pathology/table_timeline_gleason_scores.tsv'
fname_path = config_cdm.fname_pathology

In [ ]:
obj = obj_minio.load_obj(path_object=fname_path)
df_path = pd.read_csv(obj, sep='\t')

In [ ]:
obj = obj_minio.load_obj(path_object=fname_pdl1)
df_pdl1 = pd.read_csv(obj, sep='\t')

In [ ]:
obj = obj_minio.load_obj(path_object=fname_gleason)
df_gleason = pd.read_csv(obj, sep='\t')
df_gleason = convert_to_int(df=df_gleason, list_cols=['Gleason'])
df_gleason = df_gleason.drop(columns=['Accession Number'])

In [ ]:
_col_order_gleason = [
    'MRN', 
    'START_DATE', 
    'STOP_DATE', 
    'EVENT_TYPE', 
    'SUBTYPE', 
    'SOURCE', 
    'GLEASON_SCORE'
]

df_gleason = df_gleason.rename(columns={'Path Procedure Date': 'START_DATE', 'Gleason':'GLEASON_SCORE'})
df_gleason = df_gleason.assign(STOP_DATE='')
df_gleason = df_gleason.assign(EVENT_TYPE='Diagnosis')
df_gleason = df_gleason.assign(SUBTYPE='Gleason Score')
df_gleason = df_gleason.assign(SOURCE='CDM')
df_gleason = df_gleason[_col_order_gleason]

df_gleason;


In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('', '..', '..', 'cdm-utilities')))
from data_classes_cdm import CDMProcessingVariables as c_dar
from pathology_extract_pdl1 import PathologyExtractPDL1

## Constants
col_text = 'PATH_REPORT_NOTE'
fname_save = c_dar.fname_path_pdl1
fname_path = c_dar.fname_path_clean
fname_minio_env = c_dar.minio_env

# Extract PD-L1
obj_p = PathologyExtractPDL1(minio_env=fname_minio_env, 
                             fname=fname_path, 
                             col_text=col_text)

df_pdl1 = obj_p.return_extraction()

In [ ]:
# df_pdl1[['MRN', 'ACCESSION_NUMBER', 
def list_length(df, col):
    series_len = df.loc[df[col].notnull(), col].astype(str).apply(lambda x: eval(x)).apply(lambda x: len(x))
    
    return series_len
    

In [ ]:
df_pdl1

In [ ]:
series_len = df_pdl1_anno.loc[df_pdl1_anno[col_current].notnull(), col_current].apply(lambda x: len(x))

In [ ]:
pct_current = df_pdl1_anno.loc[index_current, col_current]
pct_current

In [ ]:
cols_extract = ['PDl1_PERCENTAGE', 'PDl1_CPS_1', 'PDl1_CPS_2', 'PDl1_TPS_1', 'PDl1_TPS_2', 'PDl1_IPS']
df_pdl1_anno = df_pdl1.copy()

for col_current in cols_extract:
    new_col = col_current + '_EST'
    length_current = df_pdl1_anno.loc[df_pdl1_anno[col_current].notnull(), col_current].apply(lambda x: len(x))
    index_current = length_current[length_current >0].index
    pct_current = df_pdl1_anno.loc[index_current, col_current].apply(lambda x: x[0]).str.strip()
    pct_current = pct_current.str.replace('%', '').str.replace(' ', '').str.replace('<1', '0').str.replace('>1', '1').str.replace('>=1', '1').str.replace('>', '').str.replace('<', '').str.replace('=', '')

    # Takes second value in range
    pct_range = pct_current[pct_current.str.contains('-')].str.split('-').apply(lambda x: x[1])
    pct_current[pct_current.str.contains('-')] = pct_range


    pct_current = pd.to_numeric(pct_current, errors='coerce')
    df_pdl1_anno[new_col] = pct_current
    
PDL1_POSITIVE = (df_pdl1_anno['PDl1_PERCENTAGE_EST'] >= 1) | (df_pdl1_anno['PDl1_TPS_1_EST'] >= 1) | (df_pdl1_anno['PDl1_TPS_2_EST'] >= 1)
PDL1_POSITIVE_NULL = df_pdl1_anno['PDl1_PERCENTAGE_EST'].isnull() & df_pdl1_anno['PDl1_TPS_1_EST'].isnull() & df_pdl1_anno['PDl1_TPS_2_EST'].isnull()
df_pdl1_anno['PDL1_POSITIVE'] = PDL1_POSITIVE
df_pdl1_anno = df_pdl1_anno.replace({'PDL1_POSITIVE': {True: 'Yes', False: 'No'}})
df_pdl1_anno.loc[PDL1_POSITIVE_NULL, 'PDL1_POSITIVE'] = np.NaN
df_pdl1_anno

In [ ]:
obj_site_summary = MetastaticSiteSummary(fname_minio_env=fname_minio_env, fname_timeline=fname_dx_timeline_met)

In [ ]:
print_df_without_index(pd.DataFrame(obj_site_summary.return_summary().columns))

In [ ]:
obj_dx_timeline = cBioPortalDiagnosisTimeline(
    fname_minio_config=config_cdm.minio_env,
    fname_impact_map=fname_impact_map, 
    fname_demo=fname_demo, 
    fname_dx=fname_dx, 
    fname_cbio=fname_cbio, 
    fname_rad_pred=fname_rad_pred,
    fname_rad_rpt=fname_rad_rpt
)

In [ ]:
df_prim = obj_dx_timeline.primary_dx_clean_and_merge(fname_save=fname_save_dx_prim)

In [ ]:
df_mets = obj_dx_timeline.metastatic_clean_and_merge(fname_save=fname_save_dx_met)

In [ ]:
df_ln = obj_dx_timeline.lymph_node_clean_and_merge(fname_save=fname_save_dx_ln)

In [ ]:
# df_dx_timeline = obj_dx_timeline._df_dx_timeline
# df_sample_summary = obj_dx_timeline._df_sample_summary
# df_demo = obj_dx_timeline._df_demo
# df_rad_pred = obj_dx_timeline._df_rad
# df_impact = obj_dx_timeline._df_impact

# obj_met_timeline = MetastaticTimeline(
#     df_dx_timeline=df_dx_timeline, 
#     df_sample_summary=df_sample_summary, 
#     df_demo=df_demo, 
#     df_impact=df_impact,
#     fname_rad_pred=df_rad_pred
# )

# df_f = obj_met_timeline.metastatic_clean_and_merge()

In [ ]:
# df_f['ANATOMIC_LOCATION'].value_counts()
# # df_rad_pred
# df_f.sample()

In [ ]:
# df_ln = obj_met_timeline.lymph_node_clean_and_merge()

In [ ]:
# df_ln.sample()

In [ ]:
# df_dx_timeline['METASTATIC_SITE'].value_counts()